In [1]:
import scipy.io
import os
import pandas as pd

In [2]:
directory = 'data/Data/F1/mat'
counter = 1
UL_df, LL_df, JW_df, TD_df, TB_df, TT_df = [], [], [], [], [], []

for filename in sorted(os.listdir(directory)):
    if filename.endswith('.mat'):
        f = os.path.join(directory, filename)
        mat = scipy.io.loadmat(f)
        # takes the data that is stored at the following key
        data = mat['usctimit_ema_f1_{:03}_{:03}'.format(counter, counter + 4)]
        counter += 5

        # makes dataframes of the six electrodes
        UL_df.append(pd.DataFrame.from_dict(data[0][1][2]))
        LL_df.append(pd.DataFrame.from_dict(data[0][2][2]))
        JW_df.append(pd.DataFrame.from_dict(data[0][3][2]))
        TD_df.append(pd.DataFrame.from_dict(data[0][4][2]))
        TB_df.append(pd.DataFrame.from_dict(data[0][5][2]))
        TT_df.append(pd.DataFrame.from_dict(data[0][6][2]))

In [3]:
UL_df

[              0          1         2
 0     11.517622 -57.244717  0.553757
 1     11.529473 -57.236338  0.553376
 2     11.544196 -57.223581  0.568780
 3     11.545286 -57.213561  0.593878
 4     11.517394 -57.210572  0.609659
 ...         ...        ...       ...
 1826  10.224346 -56.576700  2.058842
 1827  10.219217 -56.519980  2.056851
 1828  10.213163 -56.465348  2.057998
 1829  10.178992 -56.445427  2.078448
 1830  10.109163 -56.463028  2.116755
 
 [1831 rows x 3 columns],
               0          1         2
 0     11.861230 -57.820280  2.531607
 1     11.815625 -57.836856  2.522976
 2     11.809622 -57.842722  2.541717
 3     11.839146 -57.834584  2.586242
 4     11.870585 -57.814729  2.635691
 ...         ...        ...       ...
 2060  11.517299 -57.960753  1.674321
 2061  11.512734 -57.967277  1.674140
 2062  11.523654 -57.942313  1.681953
 2063  11.542347 -57.907986  1.670150
 2064  11.560435 -57.883174  1.630995
 
 [2065 rows x 3 columns],
               0          1     